In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pyreadr
import seaborn as sns
import matplotlib.pyplot as plt
import string
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
df = pd.read_csv('voter_file.csv')
print(df.head())

/var/folders/w6/j8d1_lqx5wz_zztjb28h8rkh0000gn/T/ipykernel_92140/571683533.py:1: DtypeWarning: Columns (45,46,47,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('voter_file.csv')


      voterbase_id  version                 household_id_hash  \
0  MO-000000000015  2024_01  666dd978701b0ad828eb3c35262548ad   
1  MO-000000000025  2024_01  cb646c5b6581ea80498fa4a0525e46cd   
2  MO-000000000051  2024_01  ec0fe42f45bac7bb43d2ce89765c417c   
3  MO-000000000083  2024_01  05f243ec7fd107768c583374c8076745   
4  MO-000000000085  2024_01  f046eb6de85a25572c7ba029e14364a4   

  tsmart_exact_track_id  smartvan_id tsmart_precinct_id    sos_id  \
0       Y29454846321647          8.0                008  42017891   
1       Y29454238536941         17.0                517  46591254   
2       Y29454813432049         36.0            FLO.006  73280446   
3       Y29458488718403         58.0                105  46604287   
4       Y29454689393585         60.0                 15  20444309   

   legal_commercial_model_usage_ok  vb_deceased  vote_g2024  ...  \
0                                0            0           0  ...   
1                                1            0           

In [3]:
# # Turn object columns into strings for easier processing
# object_columns = df.select_dtypes(include=['object']).columns

# def convert_to_string(value):
#     return str(value)

# # Apply the conversion function only to mixed columns using applymap
# df[object_columns] = df[object_columns].map(convert_to_string)

print(df.dtypes.value_counts())

object     61
int64      34
float64    29
Name: count, dtype: int64


In [18]:

# def convert_to_string_column(col):
#     return col.apply(str)

# # Apply the conversion function only to mixed columns using apply
# df[object_columns] = df[object_columns].apply(lambda col: convert_to_string_column(col))

# print(df.dtypes.value_counts())

object     61
int64      34
float64    29
Name: count, dtype: int64


In [4]:
print(df.shape)

# Delete irrelevant columns
del df['voterbase_id'], df['version'], df['household_id_hash'], df['tsmart_exact_track_id'], df['smartvan_id'], df['tsmart_precinct_id'], df['sos_id'], df['tsmart_census_block_fips_2010'], df['tsmart_cd_version_2020'], df['tsmart_cd_2020'], df['tsmart_cd_version_2010'], df['tsmart_sd'], df['tsmart_sd_2010']
del df['tsmart_sd_version'], df['tsmart_sd_version_2010'], df['tsmart_hd_version'], df['tsmart_hd_version_2020'], df['tsmart_hd_version_2010'], df['vf_reg_cd_version'], df['vf_reg_cd_version_2010'], df['vf_reg_sd_2010'], df['vf_reg_sd_version'], df['vf_reg_sd_version_2020'], df['vf_reg_sd_version_2010'], df['vf_reg_hd'], df['vf_reg_hd_2010']
del df['vf_reg_hd_version_2010'], df['vf_reg_census_id'], df['vf_reg_census_id_2010'], df['vf_reg_census_id_version'], df['vf_reg_census_id_version_2010'], df['vf_reg_precinct_name'], df['vf_reg_place_name'], df['vb_dob']

(4258869, 124)


In [5]:
print(max(df['gotv_score']) , min(df['gotv_score']))
print(max(df['turnout_score']) , min(df['turnout_score']))
print(max(df['support_score']) , min(df['support_score']))
print(max(df['synthetic_persuasion_score']) , min(df['synthetic_persuasion_score']))

df['gotv_score'] = (df['gotv_score'] - np.min(df['gotv_score'])) / (np.max(df['gotv_score']) - np.min(df['gotv_score']))
df['synthetic_persuasion_score'] = (df['synthetic_persuasion_score'] - np.min(df['synthetic_persuasion_score'])) / (np.max(df['synthetic_persuasion_score']) - np.min(df['synthetic_persuasion_score']))


0.0065201295528617 -0.0058101947090944
0.97184016271401 0.0747956319413997
0.9909344186186488 0.0313679301143077
0.0227886075320771 0.0004339507877027


In [6]:
print(df.shape)
print(max(df['gotv_score']) , min(df['gotv_score']))
print(max(df['turnout_score']) , min(df['turnout_score']))
print(max(df['support_score']) , min(df['support_score']))
print(max(df['synthetic_persuasion_score']) , min(df['synthetic_persuasion_score']))

(4258869, 90)
1.0 0.0
0.97184016271401 0.0747956319413997
0.9909344186186488 0.0313679301143077
1.0 0.0


In [7]:
def map_scores(value):
    if value <= .10:
        return 0
    elif value <= .20:
        return 1
    elif value <= .30:
        return 2
    elif value <= .36:
        return 3
    elif value <= .41:
        return 4
    elif value <= .45:
        return 5
    elif value <= .47:
        return 6
    elif value <= .49:
        return 7
    elif value <= .51:
        return 8
    elif value <= .53:
        return 9
    elif value <= .55:
        return 10
    elif value <= .59:
        return 11
    elif value <= .64:
        return 12
    elif value <= .70:
        return 13
    elif value <= .75:
        return 14
    elif value <= .80:
        return 15
    elif value <= .90:
        return 16
    elif value <= 1.00:
        return 17
    else: pass

df['gotv_score'] = df['gotv_score'].apply(map_scores)
df['turnout_score'] = df['turnout_score'].apply(map_scores)
df['support_score'] = df['support_score'].apply(map_scores)
df['synthetic_persuasion_score'] = df['synthetic_persuasion_score'].apply(map_scores)


In [8]:
### Factorize the entire dataset
### Include NaNs as factor levels

# Identify columns with NaNs
columns_with_missing = df.columns[df.isnull().any()].tolist()

# Convert columns to factor variables
for col in df.columns:
    if col not in columns_with_missing:
        df[col] = pd.factorize(df[col])[0]  # Factorize non-missing values

# Handle missing values
for col in columns_with_missing:
    df[col] = pd.factorize(df[col].fillna('MISSING'))[0]  # Factorize with 'MISSING' as a category


In [30]:
# Create the 'out' or 'target' dataset

print(df.dtypes.value_counts())

# df.to_csv('cleaned.csv', index=False)

# out = df[["vote_g2022", "vote_p2022"]]
# filt = df.drop(columns = ["vote_g2022", "vote_p2022", 'vote_g2023', 'vote_g2024'])


int64    90
Name: count, dtype: int64


In [31]:
# filt.isnull().sum()
# out.isnull().sum()

# # Turn columns of filt into integers if they are float
# # for col in filt.columns:
# #     filt.loc[:, col] = filt[col].astype('Int64')

# # for col in out.columns:
# #     out.loc[:, col] = out[col].astype('Int64')

# # Print the number of unique items in each column to get a sense of "vocabulary" size
# for column in filt:
#     try: 
#         unique_vals = np.unique(df[column])
#     except:
#         unique_vals = df[column].unique()

#     nr_vals = len(unique_vals)

#     if nr_vals == 1:
#         del filt[column]
#     elif nr_vals < 10:
#         print('The number of values for feature {} :{} -- {}'.format(column, nr_vals, unique_vals))
#     elif nr_vals > 10000:
#         del filt[column]
#     else:
#         print('The number of values for feature {}:{}'.format(column, nr_vals))

# print(filt.shape)

The number of values for feature legal_commercial_model_usage_ok :2 -- [0 1]
The number of values for feature vote_g2020 :2 -- [0 1]
The number of values for feature vote_g2018 :2 -- [0 1]
The number of values for feature vote_g2016 :2 -- [0 1]
The number of values for feature vote_g2014 :2 -- [0 1]
The number of values for feature vote_g2012 :2 -- [0 1]
The number of values for feature vote_p2018 :2 -- [0 1]
The number of values for feature vote_any_primary_since_2012 :2 -- [0 1]
The number of values for feature firsttimeseen_voterid:76
The number of values for feature state_code:58
The number of values for feature tsmart_county_fips:2691
The number of values for feature tsmart_census_id_version :2 -- [0 1]
The number of values for feature tsmart_census_id_version_2010 :2 -- [0 1]
The number of values for feature housing_temp_address :2 -- [0 1]
The number of values for feature tsmart_county_name:1606
The number of values for feature tsmart_dma_id:211
The number of values for feature 

In [32]:
# columns_with_null = filt.columns[filt.isnull().any()].tolist()
# print(columns_with_null)
# filt[columns_with_null] = filt[columns_with_null].fillna('none')

# columns_with_null = out.columns[out.isnull().any()].tolist()
# if columns_with_null:
#     out[columns_with_null] = out[columns_with_null].fillna('none')

[]


In [10]:
def generate_double_strings():
    double_strings = []
    for char1 in string.ascii_lowercase:
        for char2 in string.ascii_lowercase:
            double_strings.append(char1 + char2)
    return double_strings

column_mapping_filt = dict(zip(df.columns, generate_double_strings()))
# column_mapping_out = dict(zip(out.columns, generate_double_strings()))

# Define the function to append double strings to values
def append_double_string(val, col_name, column_mapping):
    if pd.notna(val):
        return f"{column_mapping[col_name]}{val}"
    else:
        return val  # Return NaN if the value is NaN

# Apply the function element-wise to the DataFrame
for col in df.columns:
    df.loc[:, col] = df[col].apply(lambda x, col_name=col: append_double_string(x, col_name, column_mapping_filt))

# for col in out.columns:
#     out.loc[:, col] = out[col].apply(lambda x, col_name=col: append_double_string(x, col_name, column_mapping_out))

# Convert all values in the DataFrame to strings
# filt = filt.astype(str)
# filt = filt.replace("<NA>", pd.NA)

# out = out.astype(str)
# out = out.replace("<NA>", pd.NA)

print(df.head())
# print(out.head())

  legal_commercial_model_usage_ok vb_deceased vote_g2024 vote_g2023  \
0                             aa0         ab0        ac0        ad0   
1                             aa1         ab0        ac0        ad0   
2                             aa1         ab0        ac0        ad0   
3                             aa1         ab0        ac0        ad0   
4                             aa1         ab0        ac0        ad0   

  vote_g2022 vote_g2021 vote_g2020 vote_g2019 vote_g2018 vote_g2016  ...  \
0        ae0        af0        ag0        ah0        ai0        aj0  ...   
1        ae1        af0        ag0        ah0        ai0        aj1  ...   
2        ae0        af0        ag0        ah0        ai1        aj0  ...   
3        ae0        af0        ag0        ah0        ai0        aj1  ...   
4        ae0        af0        ag0        ah0        ai0        aj1  ...   

  acs_pct_educ_bachelors_bucket household_income_bucket home_value_bucket  \
0                           dc0        

In [34]:
# filt.to_csv('filtered_data.csv', index=False)
# out.to_csv('targets.csv', index=False)

In [1]:
df.to_csv('factorized_data.csv', index=False)

NameError: name 'df' is not defined